In [1]:
import pandas as pd
import numpy as np

# 7.1.3 沿着轴串联

另一种结合方式被称为可互换的，比如concatenation, binding, or stacking(串联，绑定，堆叠)。Numpy中的concatenate函数可以作用于numpy 数组：

In [38]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.]])

In [39]:
np.concatenate([arr, arr], axis=1)

array([[  0.,   1.,   2.,   3.,   0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.,   4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.,   8.,   9.,  10.,  11.]])

而在pandas的对象中，比如Series和DataFrame，labeled axes（便签化的轴）能让我们做更泛化的数组串联操作。不过我们可能会有下面一些疑问：

- 如果一个对象在其他轴上的index不同，我们应不应该在这些轴上把不同的元素合并起来，或者只用交集?

- 经过串联操作后，连接的部分在输出对象里应不应该是可被识别的？

- concatenation axis（串联轴）含有的数据需要被保留吗？在很多情况下，DataFrame中一些用整数做的label（标签）其实最好在串联后被删除。

pandas中的concat函数能解决上面这些问题。这里会给出几个例子来说明。假设我们有三个Series，他们指明没有index overlap(索引重叠)：

In [40]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

调用concat，把上面的series放在一个list里，结果会把值和索引都整合在一起：

In [41]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认情况下，concat中axis=0,结果会得到一个新的而series。如果令axis=1, 结果会变成一个DataFrame（axis=1 是列）：

In [42]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


这种情况下，不会与其他轴产生重叠，效果与join中的outer join一样。你也可以通过设定join='inner'来使用交集：

In [43]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [44]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [45]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


因为join='inner'，所以f和g标签消失了。

你也可以在join_axes中指定使用哪些轴：

In [46]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


一个潜在的问题是串联的部分在结果里是不可辨识的。假设我们想在串联轴上创建一个多层级索引，我们需要用到keys参数：

In [48]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [49]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


如果是设定axis=1，那么keys会变为DataFrame的column header(列头):

In [51]:
print(s1)
print(s2)
print(s3)

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64


In [50]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


这种逻辑也可以扩展到DataFrame对象上：

In [52]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], 
                   columns=['one', 'two'])

df1

,one,two
a,0,1
b,2,3
c,4,5


In [53]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], 
                   columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [54]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

如果导入一个dict而不是list，那么dict的key会被用于上面concat中的keys选项：

In [55]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

还有其他一些选项负责多层级索引的设定（表8-3）。比如，可以给创建的axis level(轴层级)用names参数来命名：

In [56]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

最后我们关心的是，在DataFrame中，行索引（row index）没有包含相关的数据：

In [57]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,1.049308,-0.660746,1.152071,-1.447441
1,-0.484170,-0.096755,-0.815349,1.839818
2,-0.277541,0.164721,-0.012481,0.477152


In [58]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,-0.556378,-2.286601,-0.494776
1,1.152716,0.270165,-0.222289


这种情况下，可以设置ignore_index=True:

In [59]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.049308,-0.660746,1.152071,-1.447441
1,-0.484170,-0.096755,-0.815349,1.839818
2,-0.277541,0.164721,-0.012481,0.477152
3,-0.494776,-0.556378,NaN,-2.286601
4,-0.222289,1.152716,NaN,0.270165
